In [1]:
import sys
import os
import random
import json

# 1. Add the parent directory (Finetuning) to sys.path to import utils_ft
# This assumes the notebook is running in Finetuning/notebooks/
sys.path.append('..')

from utils_ft import (
    get_benchmark_data_path, 
    get_processed_data_path, 
    get_scripts_path,
    to_relative
)

# 2. Add scripts folder to path so we can import the converter
sys.path.append(get_scripts_path())

from data_converter import main as run_converter

# Define Paths using the Utils
data_in = get_benchmark_data_path()
data_out = get_processed_data_path()

print(f"Input:  {to_relative(data_in)}")
print(f"Output: {to_relative(data_out)}")

Input:  Finetuning/data/Benchmark Data
Output: Finetuning/data


In [2]:
run_converter(data_in, data_out)

Scanning /Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/Finetuning/data/Benchmark Data...

--- Processing Training Set (43 Groups) ---
> room_booking_easy_1
> equal_sum_partition
> routing_med_1
> RelayRace
> knapsack
> routing_easy_2
> graph_coloring
> wolf_goat_cabbage
  [Extraction] Querying Gemini for structure...
> prime_numbers
> hanoi_tower
> RichocetRobots
> bin_packing
> prime_sieve
> routing_easy_3
> clique
> config_easy_2
> n_queens
> routing_med_2
> config_easy_3
> teaspoon_standard
> visit_all
> room_booking_med_3
> dominating_set
> room_booking_hard_3
> max_flow
> config_med_3
> grandpa
> routing_med_3
> independent_set
> room_booking_med_2
> routing_hard_1
> sudoku
> Nomystery
> room_booking_easy_2
> subset_sum
> routing_hard_2
> labyrinth
> room_booking_hard_2
> routing_hard_3
> config_med_1
> 3_dem_matching
> room_booking_easy_3
> config_easy_1

--- Processing Validation Set (11 Groups) ---
> room_booking_med_1
> Asprilo
> config_hard_1
> config_hard_

<block>:1:28-29: error: syntax error, unexpected <VARIABLE>



In [3]:
train_file = os.path.join(data_out, "train.jsonl")

# 1. Load all lines into memory (efficient enough for small-medium datasets)
with open(train_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 2. Pick a random line
if lines:
    random_line = random.choice(lines)
    sample = json.loads(random_line)

    print(f"--- RANDOM SAMPLE (Total Examples: {len(lines)}) ---")
    
    # 3. Iterate through messages and print nicely
    for msg in sample['messages']:
        role = msg['role'].upper()
        content = msg['content']
        
        print(f"\n[{role}]:")
        print("-" * 20)
        print(content)
        print("=" * 60)
else:
    print("Dataset file is empty.")

--- RANDOM SAMPLE (Total Examples: 467) ---

[SYSTEM]:
--------------------
You are an expert in Answer Set Programming. Task: Generate a hard constraint (integrity constraint) based on the description. Output: Provide ONLY the valid Clingo code. Do not provide any explanation, comments, or markdown formatting.

[USER]:
--------------------
CONTEXT:
<<problem_description>>
An autonomous rover is tasked with collecting critical samples in a subterranean Mars base tunnel network. The mission
begins at the LaunchPad with the rover's water tank fully filled to 16 liters, which serves as both coolant and
hydraulic fluid for its drilling operations. The rover must navigate through a series of interconnected tunnels and 
stations within a strict 10-minute timeframe.
From the LaunchPad, the rover can proceed through either Tunnel1 or Tunnel2, requiring 2 and 3 liters of water 
respectively due to varying cooling needs across different thermal gradients. Tunnel1 leads to StationA, consuming 
an